In [9]:
import pandas as pd
demo = pd.read_sas('../data/raw/DEMO_L.xpt')
bmx = pd.read_sas('../data/raw/BMX_L.xpt')
paq = pd.read_sas('../data/raw/PAQ_L.xpt')

df = demo.merge(bmx, on='SEQN').merge(paq, on='SEQN')

#df.shape
df['RIAGENDR'].value_counts
#df['BMXBMI'].describe

<bound method IndexOpsMixin.value_counts of 0       1.0
1       1.0
2       2.0
3       1.0
4       2.0
       ... 
6332    2.0
6333    2.0
6334    1.0
6335    1.0
6336    2.0
Name: RIAGENDR, Length: 6337, dtype: float64>

In [ ]:
df = df[df['BMXBMI'].notna()]
df["gender"] = df["RIAGENDR"].map({1:"Male",2:"Female"})
df["obese"] = (df["BMXBMI"]>=30).astype(int)

In [23]:
df.groupby("gender").size()

gender
Female    3418
Male      2817
dtype: int64

In [30]:
df.groupby("gender")["obese"].mean()

gender
Female    0.428321
Male      0.376287
Name: obese, dtype: float64

In [39]:
features = [
    "PAD800",   # minutes moderate activity
    "PAD820",   # minutes vigorous activity
    "PAD680",   # sedentary minutes
    "RIDAGEYR"  # age
]


model_df = df[features + ["obese", "gender"]].dropna()

model_df.head


<bound method NDFrame.head of       PAD800  PAD820  PAD680  RIDAGEYR  obese  gender
0       45.0    45.0   360.0      43.0      0    Male
1       45.0    45.0   480.0      66.0      1    Male
3       30.0    30.0   180.0      34.0      1    Male
6       45.0    30.0   720.0      59.0      0    Male
8       10.0    20.0   240.0      33.0      1  Female
...      ...     ...     ...       ...    ...     ...
6318    60.0    60.0   360.0      57.0      0    Male
6322    60.0    60.0   150.0      68.0      0  Female
6323    60.0    60.0   240.0      60.0      1    Male
6327    60.0    30.0   120.0      33.0      1    Male
6330    45.0    20.0   120.0      70.0      0    Male

[2770 rows x 6 columns]>

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = model_df[features]
y = model_df["obese"]

X_train, X_test, y_train, y_test, g_train, g_test = train_test_split(
    X, y, model_df["gender"], test_size=0.3, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

preds = model.predict(X_test)

y_test.head()
print((g_train == "Male").sum())
print((g_train == "Female").sum())
print((g_test == "Male").sum())
print((g_test == "Female").sum())


988
951
428
403


In [69]:
for g in ["Male","Female"]:
    idx = g_test == g
    acc = accuracy_score(y_test[idx], preds[idx])
    print(g,acc)
    

Male 0.6752336448598131
Female 0.6401985111662531
